In [1]:
import psycopg2
import numpy as np
import pandas as pd

# 경고 처리
import warnings
warnings.filterwarnings('ignore')

# float 자리수 설정
pd.options.display.float_format = '{:.2f}'.format

# DB Connect
conn = psycopg2.connect(
    host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port=5439,
    dbname='dev',
    user='awsuser',
    password='cremaoAdmin1234qwer!!'
)
cursor = conn.cursor()

### 1. 연별 합계 sum

In [52]:
# 유동인구
# 2020
query = 'select sum(f_tot) from m1.kt_bldg_xy_info where base_ym between 202001 and 202012' # 쿼리문
cursor.execute(query) # 실행
f_tot_2020 = cursor.fetchall() # 출력
f_tot_2020 = f_tot_2020[0][0] # 값 꺼내기

# 2021
query = 'select sum(f_tot) from m1.kt_bldg_xy_info where base_ym between 202101 and 202112'
cursor.execute(query)
f_tot_2021 = cursor.fetchall()
f_tot_2021 = f_tot_2021[0][0]

# 2022
query = 'select sum(f_tot) from m1.kt_bldg_xy_info where base_ym between 202201 and 202212'
cursor.execute(query)
f_tot_2022 = cursor.fetchall()
f_tot_2022 = f_tot_2022[0][0]

In [53]:
# 거주인구
# 2020
query = 'select sum(pop) from m1.kt_bldg_xy_info where base_ym between 202001 and 202012'
cursor.execute(query)
pop_2020 = cursor.fetchall()
pop_2020 = pop_2020[0][0]

# 2021
query = 'select sum(pop) from m1.kt_bldg_xy_info where base_ym between 202101 and 202112'
cursor.execute(query)
pop_2021 = cursor.fetchall()
pop_2021 = pop_2021[0][0]


# 2022
query = 'select sum(pop) from m1.kt_bldg_xy_info where base_ym between 202201 and 202212'
cursor.execute(query)
pop_2022 = cursor.fetchall()
pop_2022 = pop_2022[0][0]

In [54]:
# 직장인구
# 2020
query = 'select sum(wpop) from m1.kt_bldg_xy_info where base_ym between 202001 and 202012'
cursor.execute(query)
wpop_2020 = cursor.fetchall()
wpop_2020 = wpop_2020[0][0]

# 2021
query = 'select sum(wpop) from m1.kt_bldg_xy_info where base_ym between 202101 and 202112'
cursor.execute(query)
wpop_2021 = cursor.fetchall()
wpop_2021 = wpop_2021[0][0]

# 2022
query = 'select sum(wpop) from m1.kt_bldg_xy_info where base_ym between 202201 and 202212'
cursor.execute(query)
wpop_2022 = cursor.fetchall()
wpop_2022 = wpop_2022[0][0]

### 1. 월별 합계 (sum)

In [2]:
# 월별 합(sum)
def getData(col, df):
    
    year_list  = pd.date_range(start='2020', end='2022', freq='YS').strftime('%Y')
    month_list = pd.date_range(start='2021.01', end='2021.12', freq='MS').strftime('%m')

    for year in year_list:

        for month in month_list:

            query = f'select sum({col}) from m1.kt_bldg_xy_info where base_ym = {year}{month}'
            cursor.execute(query)
            data = cursor.fetchall()
            df_sub = pd.DataFrame([data[0][0]], columns=[year+month]).T
            df = df.append(df_sub)

    return df

# base 데이터프레임 생성
pop = pd.DataFrame() # 거주인구
wpop = pd.DataFrame() # 직장인구
f_tot = pd.DataFrame() # 유동인구

# 월별 합 데이터프레임화
pop = getData('pop',pop) # 거주인구
wpop = getData('wpop',wpop) # 직장인구
f_tot = getData('f_tot', f_tot) # 유동인구

# 직장인구만 Nonetype으로 들어가서 따로 NaN 처리
wpop[wpop[0].isnull()] = np.nan

In [3]:
# 데이터 분포 확인
f_tot.describe()

,0
count,32
mean,4382727171089
std,367832846661
min,3618135717697
25%,4199154591963
50%,4498697527770
75%,4633358084166
max,4949080181901


### 1. pnu별 연별합

###### ver1

In [23]:
# 월별 합(sum)
def getData(year, col, df):
    
    query = f'''select substring(bd_mgt_sn, 19), sum({col})
                from   m1.kt_bldg_xy_info 
                where  base_ym between {year}01 and {year}12
                group  by substring(bd_mgt_sn, 19)'''
    cursor.execute(query)
    data = cursor.fetchall()
    df = pd.DataFrame(data)

    return df

# base 데이터프레임 생성
pop = pd.DataFrame() # 거주인구
wpop = pd.DataFrame() # 직장인구
f_tot = pd.DataFrame() # 유동인구

# 월별 합 데이터프레임화
pop_2020 = getData('2020','pop',pop) # 거주인구
wpop = getData('wpop',wpop) # 직장인구
f_tot = getData('f_tot', f_tot) # 유동인구

# 직장인구만 Nonetype으로 들어가서 따로 NaN 처리
wpop[wpop[0].isnull()] = np.nan

# 컬럼명 설정
pop.columns = ['PNU','SUM_2020']
wpop.columns = ['PNU','SUM']
f_tot.columns = ['PNU','SUM']

###### ver2

In [ ]:
# 월별 합(sum)
def getData(year, col, df):
    
    query = f'''select substring(bd_mgt_sn, 19), sum({col})
                from   m1.kt_bldg_xy_info 
                where  base_ym between {year}01 and {year}12
                group  by substring(bd_mgt_sn, 19)'''
    cursor.execute(query)
    data = cursor.fetchall()
    df = pd.DataFrame(data)

    return df

# 월별 합 데이터프레임화
df = pd.DataFrame() # 거주인구
df2 = pd.DataFrame()
df3 = pd.DataFrame()
pop_2020 = getData('2020','pop',pop)
pop_2021 = getData('2021','pop',pop1)
pop_2022 = getData('2022','pop',pop2)
DF = pd.concat([pop_2020,pop_2021,pop_2022])

# 컬럼명 설정
DF.columns = ['PNU','SUM_2020']

###### ver3

In [2]:
# 연별 합(sum)
def getData(year, col, df):
    
    query = f'''select substring(bd_mgt_sn, 19), sum({col})
                from   m1.kt_bldg_xy_info 
                where  base_ym between {year}01 and {year}12
                group  by substring(bd_mgt_sn, 19)'''
    cursor.execute(query)
    data = cursor.fetchall()
    df_sub = pd.DataFrame(data)
    df = df.append(df_sub)

    return df

# 월별 합 데이터프레임화
def toDataframe(col):
    
    DF = pd.DataFrame()
    year_list  = pd.date_range(start='2020', end='2022', freq='YS').strftime('%Y')
    
    for year in year_list:
        
        df = pd.DataFrame()
        df = getData(year,col,df)
        DF = pd.concat([DF,df], axis=1)

    # 컬럼명 설정
    # DF.columns = ['PNU',col]
    
    return DF

In [3]:
toDataframe('f_tot')

,0,1,0,1,0,1
0,6031458,29170.22,0030843,421833.93,5030970,21423.34
1,0031119,319245.37,0031529,288760.32,4031526,124138.45
2,0032722,167366.11,4031042,210622.06,3000001,871556346.16
3,3031253,48169.79,3031356,32929.20,6031445,23359.75
4,5030922,52982.77,8030857,99330.66,1031315,114166.54
...,...,...,...,...,...,...
539869,5207677,179.85,NaN,NaN,NaN,NaN
539870,1070011,2201.36,NaN,NaN,NaN,NaN
539871,9402925,1668.13,NaN,NaN,NaN,NaN
539872,7024277,12010.46,NaN,NaN,NaN,NaN


In [7]:
pop

NameError: name 'pop' is not defined

### 2. 전년대비 증감율

In [129]:
# 전년 대비 증감율 
def perYearIncrease(col, df, num):
    
    # 변수 생성
    df[col] = 0 # 비율 컬럼
    year_list  = pd.date_range(start='2020', end='2022', freq='YS').strftime('%Y')
    month_list = pd.date_range(start='2021.01', end='2021.12', freq='MS').strftime('%m')

    # 전년 대비 값 비교
    for month in month_list:

        for i in range(len(df)):
            
            # 비교 날짜 값 설정
            val_2020 = (df.loc[year_list[0]+month,0])
            val_2021 = (df.loc[year_list[1]+month,0])
            val_2022 = (df.loc[year_list[2]+month,0])

            # 2020년 대비 2021년
            if (round((val_2021 - val_2020)/val_2020*100,2) >= num) == True:
                df.loc[year_list[1]+month,col] = 1
            else:
                df.loc[year_list[1]+month,col] = 0
                
            # 2021년 대비 2022년
            if (round((val_2022 - val_2021)/val_2021*100,2) >= num) == True:
                df.loc[year_list[2]+month,col] = 1
            else:
                df.loc[year_list[2]+month,col] = 0
    return df

# 실행
pop = perYearIncrease('pop', pop, 4.3) # 거주인구 : 4.3%
wpop = perYearIncrease('wpop', wpop, 2.9) # 직장인구 : 2.9%
f_tot = perYearIncrease('f_tot', f_tot, 17) # 유동인구 : 17%

In [130]:
# 비율 파악 (0 대비 1의 비율)
# 거주인구
round(len(pop[pop['pop']==1]) / len(pop[pop['pop']==0])*100,2)

9.09

In [131]:
# 직장인구
round(len(wpop[wpop['wpop']==1]) / len(wpop[wpop['wpop']==0])*100,2)

9.09

In [132]:
# 유동인구
round(len(f_tot[f_tot['f_tot']==1]) / len(f_tot[f_tot['f_tot']==0])*100,2)

12.5

In [133]:
# NaN 처리
pop   = pop.drop(pop[pop[0].isnull()].index, axis=0) # 거주인구
wpop  = wpop.drop(wpop[wpop[0].isnull()].index, axis=0) # 직장인구
f_tot = f_tot.drop(f_tot[f_tot[0].isnull()].index, axis=0) # 유동인구

# 2020 데이터 삭제 (계산식에만 쓰임)
pop = pop.drop(pop[pop.index.str.contains('2020')].index, axis=0) # 거주인구
wpop  = wpop.drop(wpop[wpop.index.str.contains('2020')].index, axis=0) # 직장인구
f_tot = f_tot.drop(f_tot[f_tot.index.str.contains('2020')].index, axis=0) # 유동인구